# TP1 - Thiago de Assis Lima

**Importante:** Caso você esteja visualizando esse notebook via github pages, só tem um arquivo de exemplo carregado. Para ver outros exemplos, seria necessário rodar o notebook com o upload de outros arquivos.

Além disso, é importante que o arquivo tenha os vértices todos numa linha só. (alguns exemplos do site estavam um vértice por linha).

In [281]:
FILE_NAME = "/content/min-20-1.pol"

In [282]:
class Polygon:
  def __init__(self, file_name):
    self.read_file(file_name)

  def read_file(self, file_name):
    def get_point(point_str):
      point = point_str.split("/")
      return int(point[0]) / int(point[1])

    file = open(file_name, 'r')
    line = file.readline().strip()
    parts = line.split()
    self.num_vertices = int(parts[0])
    self.vertices = []
    for i in range(1, len(parts), 2):
      first_point = get_point(parts[i])
      second_point = get_point(parts[i+1])
      self.vertices.append((first_point, second_point))

  def find_adjacent_vertices(self, vertex):
    adjacent_vertices = []
    for i in range(len(self.vertices)):
      if self.vertices[i] == vertex:
        if i == 0:
          adjacent_vertices.append(self.vertices[-1])
        else:
          adjacent_vertices.append(self.vertices[i - 1])
        adjacent_vertices.append(self.vertices[(i + 1) % len(self.vertices)])
        break
    return adjacent_vertices

In [283]:
class Triangle:
  def __init__(self, a, b, c, is_ear = False, is_convex = False, is_triangle = False):
    self.a = a
    self.b = b
    self.c = c
    self.is_ear = is_ear
    self.is_convex = is_convex
    self.is_triangle = is_triangle

In [284]:
def direction(pi, pj, pk):
    return (pk[0] - pi[0]) * (pj[1] - pi[1]) - (pj[0] - pi[0]) * (pk[1] - pi[1])

def are_collinear(p1, p2, p3):
  x1, y1 = p1
  x2, y2 = p2
  x3, y3 = p3
  return (y2 - y1) * (x3 - x2) == (y3 - y2) * (x2 - x1)

def on_segment(p1, p2, p3):
    return min(p1[0], p2[0]) <= p3[0] <= max(p1[0], p2[0]) and min(p1[1], p2[1]) <= p3[1] <= max(p1[1], p2[1])

def segments_intersect(p1, p2, p3, p4):
  d1 = direction(p3, p4, p1)
  d2 = direction(p3, p4, p2)
  d3 = direction(p1, p2, p3)
  d4 = direction(p1, p2, p4)

  if((d1 > 0 and d2 < 0) or (d1 < 0 and d2 > 0)) and ((d3>0 and d4 < 0) or (d3 < 0 and d4 >0)):
    return 1
  elif d1 == 0 and are_collinear(p3, p4, p1):
    return 1
  elif d2 == 0 and are_collinear(p3, p4, p2):
    return 1
  elif d3 == 0 and are_collinear(p1, p2, p3):
    return 1
  elif d4 == 0 and are_collinear(p1, p2, p4):
    return 1
  else:
    return 0

def check_is_triangle(p1, p2, p3):
  x1, y1 = p1
  x2, y2 = p2
  x3, y3 = p3
  area = x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)

  if area == 0:
    return False
  else:
    return True

def check_in_triangle(point, triangle):
    a, b, c = triangle.a, triangle.b, triangle.c
    p = point

    # Cálculo das direções
    d1 = direction(a, b, p)
    d2 = direction(b, c, p)
    d3 = direction(c, a, p)

    # Verificar se o ponto está no mesmo lado de cada segmento do triângulo
    has_neg = (d1 < 0) or (d2 < 0) or (d3 < 0)
    has_pos = (d1 > 0) or (d2 > 0) or (d3 > 0)

    # Se o ponto está dentro do triângulo
    if not (has_neg and has_pos):
        return True

    # Verificar se o ponto está em um dos segmentos do triângulo
    if d1 == 0 and on_segment(a, b, p):
        return True
    if d2 == 0 and on_segment(b, c, p):
        return True
    if d3 == 0 and on_segment(c, a, p):
        return True

    return False


def check_convex(vertex1, vertex2, vertex3):
  x1, y1 = vertex1
  x2, y2 = vertex2
  x3, y3 = vertex3
  cross_product = (x2 - x1) * (y3 - y2) - (y2 - y1) * (x3 - x2)
  return cross_product >= 0


def check_ear(vertex1, vertex2, vertex3, vertices):
  triangle = Triangle(vertex1, vertex2, vertex3)
  for vertex in vertices:
    if vertex not in (vertex1, vertex2, vertex3):
      if check_in_triangle(vertex, triangle):
        return False
  return True

def ear_clipping(polygon):
  triangles = []
  vertices = polygon.vertices.copy()

  while len(vertices) > 3:
    for i in range(1, len(vertices)):
      is_ear = False
      is_triangle = False
      is_convex = False
      vertex1 = vertices[i-1]
      vertex2 = vertices[i]
      vertex3 = vertices[(i+1) % len(vertices)]

      if(check_is_triangle(vertex1, vertex2, vertex3)):
        is_triangle = True
        if (check_convex(vertex1, vertex2, vertex3)):
          is_convex = True
          if (check_ear(vertex1, vertex2, vertex3, vertices)):
            is_ear = True
            vertices.remove(vertex2)
      triangle = Triangle(vertex1, vertex2, vertex3, is_ear, is_convex, is_triangle)
      triangles.append(triangle)
      if (is_ear):
        break

  if (len(vertices) == 3):
    triangle = Triangle(vertices[0], vertices[1], vertices[2], True, True, True)
    triangles.append(triangle)
  return triangles


In [285]:
import plotly.graph_objs as go

def plot_polygon(polygon):
  vertices = polygon.vertices
  fig_p = go.Figure()
  for vertex in vertices:
    x_values = [vertex[0] for vertex in vertices]
    y_values = [vertex[1] for vertex in vertices]

    # Fecha o polígono entre o ultimo e primeiro vertices
    x_values.append(vertices[0][0])
    y_values.append(vertices[0][1])

    fig_p = go.Figure(data=[go.Scatter(x=x_values, y=y_values, mode='lines+markers+text', name='Polygon')])

    fig_p.update_layout(
      title='Polygon',
      xaxis=dict(title='X'),
      yaxis=dict(title='Y'),
      showlegend=True
    )
    return fig_p

polygon = Polygon(FILE_NAME)
triangles = ear_clipping(polygon)
count = 0
for t in triangles:
  if(t.is_ear):
    count += 1

fig_p = plot_polygon(polygon)
fig_p.show()


Para triangulizar o polígono, estamos usando a estratégia de cortes de orelha. Basicamente, devemos verificar se três vertíces consecutivos (nas posições i-1, i, i+1) formam um triângulo, se sim: então devemos verificar se esse triângulo é convexo, ou seja, está dentro do polígono. Caso positivo, então devemos agora verificar se existe qualquer outro ponto do polígono dentro desse triângulo. Se não existir, então podemos falar que esse triângulo forma uma orelha, nesse caso se remove a orelha (ponto i) e calcula tudo de novo, até que resta apenas 3 vértices do polígono, formando a última orelha.

In [286]:
import plotly.graph_objects as go

DURATION_STEP = 3000
# Inicialize a figura
fig_dict = {
  "data": [],
  "layout": {},
  "frames": []
}

# Configure o layout
fig_dict["layout"]["xaxis"] = {"title": "X"}
fig_dict["layout"]["yaxis"] = {"title": "Y"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
  {
    "buttons": [
      {
        "args": [None, {"frame": {"duration": DURATION_STEP, "redraw": True},
                        "fromcurrent": True, "transition": {"duration": 300,
                                                            "easing": "quadratic-in-out"}}],
        "label": "Play",
        "method": "animate"
      },
      {
        "args": [[None], {"frame": {"duration": 0, "redraw": True},
                          "mode": "immediate",
                          "transition": {"duration": 0}}],
        "label": "Pause",
        "method": "animate"
      }
    ],
    "direction": "left",
    "pad": {"r": 10, "t": 87},
    "showactive": False,
    "type": "buttons",
    "x": 0.1,
    "xanchor": "right",
    "y": 0,
    "yanchor": "top"
  }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
      "font": {"size": 20},
      "prefix": "Passo: ",
      "visible": True,
      "xanchor": "left",
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# Crie os dados iniciais para o polígono
polygon_x = [vertex[0] for vertex in polygon.vertices] + [polygon.vertices[0][0]]
polygon_y = [vertex[1] for vertex in polygon.vertices] + [polygon.vertices[0][1]]

data_dict = {
  "x": polygon_x,
  "y": polygon_y,
  "mode": "lines",
  "name": "Polygon",
  "line": {"color": "black"}
}
data_dict_2 = {
  "x": polygon_x,
  "y": polygon_y,
  "mode": "lines",
  "name": "Triangle",
  "line": {"color": "black"}
}
fig_dict["data"].append(data_dict)
fig_dict["data"].append(data_dict_2)

vertices = polygon.vertices.copy()
for i, triangle in enumerate(triangles):
  polygon_x = [vertex[0] for vertex in vertices] + [vertices[0][0]]
  polygon_y = [vertex[1] for vertex in vertices] + [vertices[0][1]]

  x_coords = [triangle.a[0], triangle.b[0], triangle.c[0], triangle.a[0]]
  y_coords = [triangle.a[1], triangle.b[1], triangle.c[1], triangle.a[1]]

  color = "blue" if triangle.is_ear else "red"

  frame = {
    "data": [
      {"x": polygon_x, "y": polygon_y, "mode": "lines", "name": "Polygon", "line": {"color": "black"}},
      {"x": x_coords, "y": y_coords, "mode": "lines+markers", "name": f"Triangle", "line": {"color": color}}
    ],
    "name": f"frame{i}"
  }

  if(triangle.is_ear):
    vertices.remove(triangle.b)

  annotations = []
  for x, y in zip(x_coords, y_coords):
    annotations.append(
      dict(
        x=x,
        y=y,
        xref="x",
        yref="y",
        text=f"({x}, {y})",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-20
      )
    )
  title = ""
  if (triangle.is_ear):
    title = "Triangulo formou uma orelha, pois satisfez as condicoes necessarias"
  else:
    title = "Triangulo nao formou uma orelha. Motivos: "
    if (triangle.is_convex == False):
      title += "Triangulo nao eh convexo "
    elif triangle.is_triangle == False:
      title += "Os tres pontos nao formam um triangulo"
    else:
      title += "Existe pelo menos um vertice dentro do triangulo ou entao no segmento de uma de suas arestas"

  frame["layout"] = {"annotations": annotations, "title": title}

  fig_dict["frames"].append(frame)

  slider_step = {"args": [
    [f"frame{i}"],
    {"frame": {"duration": 300, "redraw": True},
      "mode": "immediate",
      "transition": {"duration": 300}}
  ],
    "label": f"{i}",
    "method": "animate"}
  sliders_dict["steps"].append(slider_step)

vertices = polygon.vertices.copy()

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()


In [287]:
def plot_triangles(triangles):
  count = 0
  fig_t = go.Figure()
  for i in range(0, len(triangles)):
    triangle = triangles[i]
    if(triangle.is_ear):
      count += 1
      x_values = [triangle.a[0], triangle.b[0], triangle.c[0], triangle.a[0]]
      y_values = [triangle.a[1], triangle.b[1], triangle.c[1], triangle.a[1]]
      fig_t.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', fill='toself', name=f'Triangle {count}'))

  fig_t.update_layout(
    title='Polígono com todas os triangulos de orelha',
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'),
    showlegend=False
  )
  return fig_t

triangles = ear_clipping(polygon)
fig_t = plot_triangles(triangles)

fig_t.show()

In [288]:
import math

all_colors = {"black", "purple", "green"}

class PointColor:
  def __init__(self, x, y, color):
    self.x = x
    self.y = y
    self.color = color

def vertex_in_points(vertex, points):
  for point in points:
    if(point.x == vertex[0] and point.y == vertex[1]):
      return point
  return None

def triangle_in_points(triangle, points):
  count = 0
  a, b, c = triangle.a, triangle.b, triangle.c
  for point in points:
    if(point.x == a[0] and point.y == a[1]):
      count += 1
    elif(point.x == b[0] and point.y == b[1]):
      count += 1
    elif(point.x == c[0] and point.y == c[1]):
      count += 1
  return count == 3


def is_equal(a1, a2):
  return a1[0] == a2[0] and a1[1] == a2[1]

ear_triangles = []
for triangle in triangles:
  if(triangle.is_ear):
    ear_triangles.append(triangle)

def get_shared_diagonal(triangle1, triangle2):
  a1, b1, c1 = triangle1.a, triangle1.b, triangle1.c
  a2, b2, c2 = triangle2.a, triangle2.b, triangle2.c
  shared = []

  if is_equal(a1, a2):
    shared.append(a1)
  elif is_equal(a1, b2):
    shared.append(a1)
  elif is_equal(a1, c2):
    shared.append(a1)

  if is_equal(b1, a2):
    shared.append(b1)
  elif is_equal(b1, b2):
    shared.append(b1)
  elif is_equal(b1, c2):
    shared.append(b1)

  if is_equal(c1, a2):
    shared.append(c1)
  elif is_equal(c1, b2):
    shared.append(c1)
  elif is_equal(c1, c2):
    shared.append(c1)

  return shared

points = []

for i in range(len(ear_triangles)):
  triangle_1 = ear_triangles[i]
  found_diagonal = False

  t2_index = i+1
  if (i+1 == len(ear_triangles)):
    t2_index = 0
  triangle_2 = ear_triangles[t2_index]
  diagonal = get_shared_diagonal(triangle_1, triangle_2)

  if len(diagonal) == 2:
    found_diagonal = True

    # Se a diagonal estiver colorida, entao só falta colocar a cor faltante nos vertices que nao tiveram coloracao
    p1 = vertex_in_points(diagonal[0], points)
    p2 = vertex_in_points(diagonal[1], points)
    if (p1 != None and p2 != None):
      missing_color = (all_colors - {p1.color, p2.color}).pop()

      for vertex in [triangle_1.a, triangle_1.b, triangle_1.c]:
        if vertex not in diagonal:
          if vertex_in_points(vertex, points) == None:
            points.append(PointColor(vertex[0], vertex[1], missing_color))

      for vertex in [triangle_2.a, triangle_2.b, triangle_2.c]:
        if vertex not in diagonal:
          if vertex_in_points(vertex, points) == None:
            points.append(PointColor(vertex[0], vertex[1], missing_color))
      continue

    if (vertex_in_points(diagonal[0], points) == None):
      points.append(PointColor(diagonal[0][0], diagonal[0][1], 'purple'))

    if (vertex_in_points(diagonal[1], points) == None):
      points.append(PointColor(diagonal[1][0], diagonal[1][1], 'green'))

    # Adiciona o outro ponto do triângulo 1 com cor preta
    for vertex in [triangle_1.a, triangle_1.b, triangle_1.c]:
      if vertex not in diagonal:
        if (vertex_in_points(vertex, points) == None):
          points.append(PointColor(vertex[0], vertex[1], 'black'))

    # Adiciona o outro ponto do triângulo 2 com cor preta
    for vertex in [triangle_2.a, triangle_2.b, triangle_2.c]:
      if vertex not in diagonal:
        if (vertex_in_points(vertex, points) == None):
          points.append(PointColor(vertex[0], vertex[1], 'black'))

black_points = 0
green_points = 0
purple_points = 0

for point in points:
  if point.color == 'black':
    black_points += 1
  elif point.color == 'green':
    green_points += 1
  elif point.color == 'purple':
    purple_points += 1

min_color = 0
min_points = 0

if (black_points == math.floor(polygon.num_vertices/3)):
  min_color = 'preto'
  min_points = black_points
elif(green_points ==  math.floor(polygon.num_vertices/3)):
  min_color = 'verde'
  min_points = green_points
else:
  min_color = 'roxo'
  min_points = purple_points


7 6 7


In [289]:
triangles = ear_clipping(polygon)

def plot_points(points):
  data = []
  for triangle in ear_triangles:
    data.append(go.Scatter(x=[triangle.a[0], triangle.b[0], triangle.c[0], triangle.a[0]], y=[triangle.a[1], triangle.b[1], triangle.c[1], triangle.a[1]], mode="lines", line=dict(color="gray")))
  for point in points:
    data.append(go.Scatter(x=[point.x], y=[point.y], mode="markers", marker=dict(color=point.color, size=10)))
  fig_p = go.Figure(
      data=data
  )
  # Atualiza o layout do gráfico
  fig_p.update_layout(
      title=f'A-Coloração. Basta adicionar cameras nos vertices de cor {min_color}, resultando em {min_points} cameras',
      xaxis=dict(title='X'),
      yaxis=dict(title='Y'),
      showlegend=False
  )

  return fig_p

fig_p = plot_points(points)
fig_p.show()


##A 3-Coloração foi feita da seguinte forma:
1.   A cada 2 triângulos consecutivos, veja se eles possuem uma diagonal compartilhada; Se sim, então cada vertice da diagonal vai ser receber uma cor verde e o outro roxo. Os dois vértices restantes dos triângulos receberam a cor preta.
2.   Na próxima iteração, se a diagonal já estiver colorida, entao basta colocar a cor que não está presente na diagonal no vértice faltante.
3. No fim, uma das cores terá tamanho n/3, consequentemente representando as cameras.

